In [1]:
from binance import Client
import requests
from dotenv import load_dotenv
import os
import ta
import pandas as pd
from time import sleep
from binance.exceptions import BinanceAPIException
import time

In [2]:
load_dotenv()

True

In [3]:
api_key = os.getenv("BINANCE_API_KEY")
api_secret = os.getenv("BINANCE_SECRET_KEY")

In [4]:
client = Client(api_key, api_secret)

In [5]:
pd.DataFrame(client.get_historical_klines('BTCBUSD', '1m', '10m ago UTC'))

,0,1,2,3,4,5,6,7,8,9,10,11
0,1639016820000,49864.00000000,49914.27000000,49850.00000000,49909.28000000,10.33418000,1639016879999,515361.48767450,307,4.84832000,241771.44032470,0
1,1639016880000,49904.28000000,49919.98000000,49860.01000000,49915.62000000,9.27519000,1639016939999,462734.02047930,366,3.32865000,166082.12081450,0
2,1639016940000,49915.62000000,49918.00000000,49880.01000000,49902.19000000,4.77845000,1639016999999,238477.45811350,247,2.17831000,108708.41627700,0
3,1639017000000,49902.02000000,49902.02000000,49800.21000000,49894.12000000,7.87024000,1639017059999,392197.47136150,393,2.84896000,141963.26052400,0
4,1639017060000,49894.12000000,49949.98000000,49885.34000000,49922.39000000,22.58290000,1639017119999,1127321.59164670,586,17.07002000,852091.80780960,0
5,1639017120000,49924.67000000,49959.18000000,49920.28000000,49959.18000000,5.58354000,1639017179999,278796.53338820,248,1.90908000,95335.77958300,0
6,1639017180000,49959.18000000,49972.96000000,49925.76000000,49939.60000000,3.33350000,1639017239999,166509.04693390,249,1.90289000,95052.54281170,0
7,1639017240000,49939.61000000,49949.14000000,49895.42000000,49920.00000000,3.25793000,1639017299999,162650.39366720,215,1.35183000,67487.88053850,0
8,1639017300000,49919.99000000,49966.29000000,49909.42000000,49951.10000000,5.86259000,1639017359999,292764.50896640,274,2.32765000,116234.96254880,0
9,1639017360000,49951.00000000,49961.31000000,49925.88000000,49945.93000000,0.99469000,1639017419999,49679.72279030,107,0.53384000,26662.17216070,0


In [8]:
def getdata(symbol):
    try:
        frame = pd.DataFrame(client.get_historical_klines(symbol, '1m', '10m UTC'))
    except BinanceAPIException as e:
        print(e)
        sleep(60)
        frame = pd.DataFrame(client.get_historical_klines(symbol, '1m', '10m UTC'))
    frame = frame.iloc[:, :6]
    frame.columns = ['Time','Open', 'High','Low','Close','Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame = frame.astype(float)
    return frame

In [9]:
getdata('BTCBUSD')

,Open,High,Low,Close,Volume
Time,,,,,
2021-12-09 02:27:00,49864.00,49914.27,49850.00,49909.28,10.33418
2021-12-09 02:28:00,49904.28,49919.98,49860.01,49915.62,9.27519
2021-12-09 02:29:00,49915.62,49918.00,49880.01,49902.19,4.77845
2021-12-09 02:30:00,49902.02,49902.02,49800.21,49894.12,7.87024
2021-12-09 02:31:00,49894.12,49949.98,49885.34,49922.39,22.58290
2021-12-09 02:32:00,49924.67,49959.18,49920.28,49959.18,5.58354
2021-12-09 02:33:00,49959.18,49972.96,49925.76,49939.60,3.33350
2021-12-09 02:34:00,49939.61,49949.14,49895.42,49920.00,3.25793
2021-12-09 02:35:00,49919.99,49966.29,49909.42,49951.10,5.86259


In [10]:
def tradingstrat(symbol, qty, open_position = False):
    while True:
        df = getdata(symbol)
        print(f'current close is $' +str(df.Close.iloc[-1]))
        if not open_position:
            if ta.trend.macd_diff(df.Close).iloc[-1] > 0 \
            and ta.trend.macd_diff(df.Close).iloc[-2] < 0 \
            and ta.trend.ema_indicator(df.Close, window=13) > ta.trend.ema_indicator(df.Close, window=62) \
            and ta.momentum.rsi(df.Close, window=14) > 50:
                order = client.create_order(symbol=symbol,
                                               side='BUY',
                                               type='MARKET',
                                               quantity=qty)
                print(order)
                open_position = True
                buyprice = float(order['fills'][0]['price'])
                break 
                
        if open_position:
            while True:
                df = getdata(symbol)
                
                if ta.trend.ema_indicator(df.Close, window=13) < ta.trend.ema_indicator(df.Close, window=62) \
                | df.Close[-1] <= buyprice * 0.90 or df.Close >= 2 * buyprice:
                    order = client.create_order(symbol=symbol,
                                               side='BUY',
                                               type='MARKET',
                                               quantity=qty) 
                    print(order)
                    sellprice = float(order['fills'][0]['price'])
                    print(f'profit = {(sellprice - buyprice)/buyprice}')
                    open_position = False
                    break

In [ ]:
while True:
    tradingstrat('BTCBUSD', qty=0.000020)

current close is $49939.59
current close is $49939.59
current close is $49939.59
current close is $49939.59
current close is $49939.59
current close is $49939.59
current close is $49939.59
current close is $49939.59
current close is $49939.59
current close is $49939.59
current close is $49939.59
current close is $49939.59
current close is $49939.59
current close is $49940.89
current close is $49940.89
current close is $49940.89
current close is $49940.89
current close is $49940.89
current close is $49940.89
current close is $49944.13
current close is $49944.13
current close is $49944.13
current close is $49944.12
current close is $49944.12
current close is $49944.12
current close is $49944.13
current close is $49944.13
current close is $49944.13
current close is $49944.12
current close is $49944.12
current close is $49944.12
current close is $49944.12
current close is $49944.12
current close is $49944.12
current close is $49944.12
current close is $49944.12
current close is $49944.12
c